# Load Processed Data into Vector Database

This notebook loads output from data prep kit into Milvus

**Step-5 in this workflow**

![](media/rag-overview-2.png)


## Step-1: Configuration

In [1]:
from my_config import MY_CONFIG

## Step-2: Load Parquet Data

Load all  `.parquet` files in the given dir

In [2]:
import pandas as pd
import glob

print ('Loading data from : ', MY_CONFIG.OUTPUT_FOLDER_FINAL)

# Get a list of all Parquet files in the directory
parquet_files = glob.glob(f'{MY_CONFIG.OUTPUT_FOLDER_FINAL}/*.parquet')
print ("Number of parquet files to read : ", len(parquet_files))
print ()

# Create an empty list to store the DataFrames
dfs = []

# Loop through each Parquet file and read it into a DataFrame
for file in parquet_files:
    df = pd.read_parquet(file)
    print (f"Read file: '{file}'.  number of rows = {df.shape[0]}")
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
data_df = pd.concat(dfs, ignore_index=True)

print (f"\nTotal number of rows = {data_df.shape[0]}")

Loading data from :  output/output_final
Number of parquet files to read :  2

Read file: 'output/output_final/attention.parquet'.  number of rows = 27
Read file: 'output/output_final/granite.parquet'.  number of rows = 33

Total number of rows = 60


In [3]:

## Shape the data

MY_CONFIG.EMBEDDING_LENGTH =  len(data_df.iloc[0]['embeddings'])
print ('embedding length: ', MY_CONFIG.EMBEDDING_LENGTH)

# rename 'embeddings' columns as 'vector' to match default schema
# if 'vector' not in data_df.columns and 'embeddings' in data_df.columns:
#     data_df = data_df.rename( columns= {'embeddings' : 'vector'})
# if 'text' not in data_df.columns and 'contents' in data_df.columns:
#     data_df = data_df.rename( columns= {'contents' : 'text'})

data_df = data_df.rename( columns= {'embeddings' : 'vector', 'contents' : 'text'})

print (data_df.info())
data_df.head(3)

embedding length:  384
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   filename            60 non-null     object 
 1   num_pages           60 non-null     int64  
 2   num_tables          60 non-null     int64  
 3   num_doc_elements    60 non-null     int64  
 4   document_hash       60 non-null     object 
 5   ext                 60 non-null     object 
 6   hash                60 non-null     object 
 7   size                60 non-null     int64  
 8   date_acquired       60 non-null     object 
 9   pdf_convert_time    60 non-null     float64
 10  source_filename     60 non-null     object 
 11  removed             60 non-null     object 
 12  source_document_id  60 non-null     object 
 13  text                60 non-null     object 
 14  document_id         60 non-null     object 
 15  vector              60 non-null     

,filename,num_pages,num_tables,num_doc_elements,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,removed,source_document_id,text,document_id,vector
0,attention.pdf,15,6,147,2949302674760005271,pdf,f1f600333e46c5d7e23f5a110a903ee38aab0bf7047eca...,46040,2025-02-19T12:31:15.922331,47.548788,attention.pdf,[],53826e81-cfa3-4118-ba56-6ad6b5eb3dd5,"Provided proper attribution is provided, Googl...",40364b6813455711d85ac8fb680212f946dd00b2f59f31...,"[-0.050560787, 0.05136596, 0.008616892, 0.0074..."
1,attention.pdf,15,6,147,2949302674760005271,pdf,f1f600333e46c5d7e23f5a110a903ee38aab0bf7047eca...,46040,2025-02-19T12:31:15.922331,47.548788,attention.pdf,[],53826e81-cfa3-4118-ba56-6ad6b5eb3dd5,## Attention Is All You Need\n\nAshish Vaswani...,45e678f43369d5fa127105b7cca6a6e4dd4deed6422185...,"[-0.018660069, 0.055049, -0.023300758, 0.04501..."
2,attention.pdf,15,6,147,2949302674760005271,pdf,f1f600333e46c5d7e23f5a110a903ee38aab0bf7047eca...,46040,2025-02-19T12:31:15.922331,47.548788,attention.pdf,[],53826e81-cfa3-4118-ba56-6ad6b5eb3dd5,## Abstract\n\nThe dominant sequence transduct...,590629323f9d88598a80846d1df6a83d0ad6ac53efe278...,"[0.03432396, 0.00844538, 0.06348668, -0.030601..."


## Step-3: Connect to Vector Database

Milvus can be embedded and easy to use.

<span style="color:blue;">Note: If you encounter an error about unable to load database, try this: </span>

- <span style="color:blue;">In **vscode** : **restart the kernel** of previous notebook. This will release the db.lock </span>
- <span style="color:blue;">In **Jupyter**: Do `File --> Close and Shutdown Notebook` of previous notebook. This will release the db.lock</span>
- <span style="color:blue;">Re-run this cell again</span>




In [4]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(MY_CONFIG.DB_URI)

print ("✅ Connected to Milvus instance:", MY_CONFIG.DB_URI)

✅ Connected to Milvus instance: ./rag_1_dpk.db


# Step-4: Create A Collection



In [5]:
# if we already have a collection, clear it first
if milvus_client.has_collection(collection_name=MY_CONFIG.COLLECTION_NAME):
    milvus_client.drop_collection(collection_name=MY_CONFIG.COLLECTION_NAME)
    print ('✅ Cleared collection :', MY_CONFIG.COLLECTION_NAME)


milvus_client.create_collection(
    collection_name=MY_CONFIG.COLLECTION_NAME,
    dimension=MY_CONFIG.EMBEDDING_LENGTH,
    metric_type="IP",  # Inner product distance
    consistency_level="Strong",  # Strong consistency level
    auto_id=True
)
print ("✅ Created collection :", MY_CONFIG.COLLECTION_NAME)


✅ Cleared collection : dpk_papers
✅ Created collection : dpk_papers


## Step-5: Insert Data into Collection

In [6]:
res = milvus_client.insert(collection_name=MY_CONFIG.COLLECTION_NAME, data=data_df.to_dict('records'))

print('inserted # rows', res['insert_count'])

milvus_client.get_collection_stats(MY_CONFIG.COLLECTION_NAME)

inserted # rows 60


{'row_count': 60}

## Step-6: Close DB Connection

Close the connection so the lock files are relinquished and other notebooks can access the db

In [7]:
milvus_client.close()

print ("✅ SUCCESS")


✅ SUCCESS


## Test your data by doing a Vector Search

See notebook [vector_search.ipynb](vector_search.ipynb)